In [1]:
# Librerías necesarias para realizar la práctica
# -----------------------------------------------------
# Manipulación y lectura de datos
import netCDF4 as nc
import pandas as pd
import numpy as np
from datetime import timedelta,datetime

# Estadistica
from scipy import stats

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Aquí voy a guardar cosas utiles que he aprendido en el ejercicio 1 pero que para la entrega no son necesarias

<span style='color:black'>The t-test quantifies the difference between the arithmetic means of the two samples. The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true. A p-value larger than a chosen threshold (e.g. 5% or 1%) indicates that our observation is not so unlikely to have occurred by chance. Therefore, we do not reject the null hypothesis of equal population means. If the p-value is smaller than our threshold, then we have evidence against the null hypothesis of equal population means.</span>

In [ ]:
# Contraste de hipotesis
# Tengo que hace un t test y p test
stats.ttest_ind(std_enero_first, std_enero_last)

from statsmodels.stats.weightstats import ztest as ztest
ztest(mean_enero_first, mean_enero_last)

In [ ]:
sns.set_theme(rc={'figure.figsize':(15,6)})
sns.boxplot(data=df_1, boxprops={"facecolor": "None", "edgecolor": sns.color_palette("Paired")[1],
                          "linewidth": 0.5},
            medianprops={'color':sns.color_palette("Set2")[0]},
            whiskerprops={'color':sns.color_palette("Paired")[1]})

sns.stripplot(data=df_1, color=sns.color_palette("Paired")[1],alpha=0.3, size=1.5)

#### Boxplot CDC

In [ ]:
sns.boxplot(data=df_1, boxprops={"facecolor": "None", "edgecolor": sns.color_palette("Paired")[1],
                          "linewidth": 0.5},
            medianprops={'color':sns.color_palette("Set2")[0]},
            whiskerprops={'color':sns.color_palette("Paired")[1]})

sns.stripplot(data=df_1, color=sns.color_palette("Paired")[1],alpha=0.3, size=1.5)